# Blackjack with AI player

In [229]:
import random

In [230]:
SUITS = ['club', 'spade', 'heart', 'diamond']
RANKS = ['2','3','4','5','6','7','8','9','10','J','Q','K','A']

# Card class keeps:
    # suit - The suit of the card [club, spade, heart, diamond]
    # rank - The rank of the card [2,3,4,5,6,7,8,9,10,J,Q,K,A]
    # value - The point value associated with the rank
    # hidden - faceDown = True ; faceUp = false
    # isAce - True if the card is an Ace
class Card(object):
    
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        
        if rank == 'A':
            self.value = 11
            self.isAce = True
        elif rank in ['J','Q','K']:
            self.value = 10
            self.isAce = False
        else:
            self.value = int(rank) # 2-10
            self.isAce = False
            
        self.hidden = False # Dealer will have one card hidden from the player
         
    def __str__(self):
        if self.hidden:
            return str(tuple('X'))
        else: 
            return str((self.suit, self.rank))
    
    def hideCard(self):
        self.hidden = True
        
    def showCard(self):
        self.hidden = False

# Deck class keeps:
    # cards - list of cards remaining in the deck
class Deck(object):
    
    def __init__(self, nDecks = 1):
        self.buildDeck(nDecks)
        self.prepareDeck()
    
    def buildDeck(self, nDecks):
        self.cards = [Card(suit,rank) for i in range(nDecks) for suit in SUITS for rank in RANKS]
    
    def prepareDeck(self):
        for i in range(7):
            self.shuffle()
            
    def shuffle(self):
        random.shuffle(self.cards)
        
    def draw(self):
        card = self.cards.pop(0)
        return card
    
    def __len__(self):
        return len(self.cards)
    
    def __str__(self):
        return str([str(card) for card in self.cards])
    
# Hand class keeps
    # hand - cards in the hand
    # bet - the bet placed on the hand
class Hand(object):
    
    def __init__(self, card = None, bet = 50):
        self.aces = 0
        if card:
            self.cards = [card]
        else:
            self.cards= []
        self.bet = bet
        
    def addCard(self, card):
        self.cards.append(card)     
            
    def removeCard(self):
        return self.cards.pop()
    
    def checkBust(self):
        if self.value > 21:
            return True
        return False
        
    @property
    def value(self):
        value = sum(card.value for card in self.cards)
        aces = sum(card.isAce for card in self.cards)  
        # Aces are initially given a value of 11, they are ammended to 1 if the total valaue is > 21
        while (value > 21) and aces: 
            value -= 10
            aces -= 1
        return value
        
    def __str__(self):
        return str([str(card) for card in self.cards])
        

In [231]:
# Player class keeps:
    # money
    # hands
class Player:
    
    def __init__(self, money = 5000, bet = 50):
        self.money = money
        self.betAmount = bet
        self.bets = []
        self.hands = [Hand()]
        
    def makeBet(self, handNumber):
        if handNumber >= len(self.bets): # new hand gets a new bet
            self.bets.append(self.betAmount)
        else: # double the bet
            self.bets[handNumber] += self.betAmount
        # remove the bet amount from players money
        self.money -= self.betAmount
            
    # True if player has enough money to make a bet, False otherwise
    def hasMoney():
        return self.money > self.betAmount
        
    def hit(self, handNumber, card):
        self.hands[handNumber].addCard(card)

    def doubleDown(self, handNumber, card):
        self.hit(handNumber, card)
        self.makeBet(handNumber)

    def split(self, handNumber, cards):
        newHand = Hand()
        newHand.addCard(self.hands[handNumber].removeCard()) # remove one card from the hand and add to newHand
        self.hands[handNumber].addCard(cards[0]) # add a card to the old hand
        self.hands.append(newHand.addCard(cards[1])) # add new hand to players hands w/ second card
        self.makeBet(len(self.hands)) # add a bet to the new hand
        
    def showHands(self):
        print("Player has %d hands: \n" % len(self.hands))
        for hand in self.hands:
            print(hand)
        print()
    
    def stats(self):
        print("Player has $%d left." % self.money)
        # possible other stats later on
        

# Dealer class keeps:
    # deck
    # hand
    # isBust
class Dealer:
    
    def __init__(self, deck, Player):
        self.deck = deck
        self.hand = Hand()
        self.Player = Player
    
    def deal(self):
        for i in range(2): # deal each player, including the dealer, 2 cards
            self.hand.addCard(self.deck.draw()) # Deal the dealer a card
            if i == 0: # dealers first card is face down
                self.hand.cards[0].hideCard() 
            self.Player.hands[0].addCard(self.deck.draw()) # Deal the player a card   
        self.Player.makeBet(0) # player makes a bet on hand
        
    def hit(self):
        self.hand.addCard(self.deck.draw())
            
    def stand(self):
        return self.hand
    
    def showHands(self):
        print('Dealers Hand:')
        print(self.hand)
    
    
def validMoves():
    return 0

def makeMoves():
    return 0

In [232]:
class Blackjack:
    
    def __init__(self, Player):
        self.Player = Player
        self.Deck = Deck(6)
        self.Dealer = Dealer(self.Deck, Player)
        self.Dealer.deal()
        
    def newHand(self):
        self.Player.hands = [Hand()]
        self.Dealer.hand = Hand()
        self.Dealer.deal()
        
    def newDeck(self):
        self.Deck = Deck(6)
       
    # hand is player's hand
    def gameStatus(self, hand):
        # player bust
        if hand.checkBust():
            return -1
        # dealer bust
        if self.Dealer.hand.checkBust():
            return 1
        # draw
        if hand.value == self.Dealer.hand.value:
            return 0
        # player win
        if hand.value > self.Dealer.hand.value:
            return 1
        # dealer win
        if hand.value < self.Dealer.hand.value:
            return -1
        

In [233]:
# returns a list of valid moves for a hand
def validMoves(player, handNumber):
    moves = ['hit','stand']
    
    # check that hand has 2 cards and the player has money to make a bet
    if len(player.hands[handNumber.cards]) == 2 & player.hasMoney(): 
        moves.append('doubleDown')
        if hand.cards[0] == hand.cards[1]:
            moves.append('split')
            
    return moves
    
def makeMove(game, handNumber, move):
    if move == 'hit':
        game.player.hit(handNumber, game.deck.draw())
        return False
    elif move == 'split':
        game.player.split(handNumber, [game.deck.draw(), game.deck.draw()])
        return False
    elif move == 'doubleDown':
        game.player.doubleDown(handNumber, game.deck.draw())
        return True
    else:
        return True
        


In [234]:
import random

# determines if a greedy move should be taken
def epsilonGreedy (epsilon, Q, player, handNumber, validMovesF):
    validMoves = validMovesF(player, handNumber)
    
    if np.random.uniform() < epsilon: # Random choice
        return random.choice(validMoves)
    else: # Greedy choice
        # Please Stand By - In Progress
        return 0

In [235]:
import numpy as np

def trainQ(nRepetitions, learningRate, epsilonDecayRate, validMovesF, makeMoveF):
    epsilon = 1.0
    
    Q = {}
    outcomes = np.zeros(nRepetitions)
    epsilons = np.zeros(nRepetitions)
    
    for gameNum in range(nRepetitions):
        epsilon *= epsilonDecayRate  # decay epsilon to move away from random choices
        epsilons[gameNum] = epsilon
        
        # create a game
        player = Player()
        blackjack = Blackjack(player)
        
        done = False
        
        # play some blackjack
        while len(blackjack.Deck) >  52: # deck is 6 decks, last deck is a buffer deck
            if blackjack.player.hasMoney(): # make sure player has the funds to play
                blackjack.newHand() # deal a hand
                handNumber = 0
            
                # For each players hand
                while handNumber < len(blackjack.player.hands):
                    done = False
                
                    # Player plays until they stand or bust
                    while not done:
                        # Determine a move for the players hand
                        move = epsilonGreedy(epsilon, Q, blackjack.player, handNumber, validMovesF)
                        
                        # make a move on the hand, done if double down or stand
                        done = makeMoveF(blackjack, handNumber, move)
                        
                        # check if bust
                        if blackjack.player.hands[handNumber].checkBust():
                            done = True
                            
                    handNumber += 1 # next hand
            else:
                break # player doesn't have the funds, game over 
        
        ### After Player plays all his hands dealer plays
        ### After Dealer plays all player hands are checked
            # payout on wins
            
        
    
    # need to add Q table stuff
        # whats my key? my cards or total value of cards?
    # should valid moves contain a blackjack?
        # or should it learn to stand on blackjack
        # i think blackjack is an automatic win
        # where should i check for blackjack
    # greedy choice
    # payout on wins (game)
    
    # test that the bets are running correctly - deal, split, doubleDown
        # check that bets are correct and money is updated properly
    # re-test valid moves
        # updated to include player has the funds required

## Testing

In [236]:
deck = Deck(1)
print(len(deck))
print(str(deck))
print(str(deck.draw()))
print(str(deck.draw()))
print(len(deck))

52
["('club', '10')", "('diamond', '4')", "('spade', '4')", "('club', 'A')", "('heart', 'Q')", "('heart', '2')", "('club', '6')", "('spade', 'Q')", "('heart', '7')", "('club', 'J')", "('club', '5')", "('club', '2')", "('diamond', 'A')", "('diamond', '5')", "('spade', 'J')", "('diamond', '8')", "('spade', '7')", "('club', 'K')", "('heart', 'A')", "('heart', '9')", "('diamond', 'Q')", "('club', '9')", "('club', '7')", "('club', '3')", "('spade', '3')", "('diamond', 'J')", "('diamond', '3')", "('heart', 'J')", "('spade', '9')", "('heart', '4')", "('spade', '5')", "('heart', '10')", "('diamond', '10')", "('diamond', 'K')", "('spade', 'K')", "('club', 'Q')", "('diamond', '6')", "('spade', '2')", "('heart', '5')", "('club', '4')", "('heart', '6')", "('heart', 'K')", "('diamond', '2')", "('spade', '8')", "('diamond', '9')", "('heart', '8')", "('heart', '3')", "('club', '8')", "('spade', '6')", "('spade', 'A')", "('spade', '10')", "('diamond', '7')"]
('club', '10')
('diamond', '4')
50


In [237]:
deck = Deck(3)
print(len(deck))
hand = Hand()
hand.addCard(deck.draw())
print(str(hand))
hand.addCard(deck.draw())
print(str(hand))
hand.addCard(deck.draw())
print(str(hand))
print(str(hand.value))

156
["('heart', '9')"]
["('heart', '9')", "('spade', '6')"]
["('heart', '9')", "('spade', '6')", "('spade', 'Q')"]
25


In [238]:
player = Player()
game = Blackjack(player)
print(len(game.Deck))
game.Player.showHands()
game.Dealer.showHands()
game.Dealer.hand.cards[0].showCard()
game.Dealer.showHands()
game.newHand()
print(len(game.Deck))
game.Player.showHands()
game.Dealer.hand.cards[0].showCard()
game.Dealer.hit()
game.Dealer.showHands()
game.newDeck()
print(len(game.Deck))

308
Player has 1 hands: 

["('heart', '10')", "('club', '4')"]

Dealers Hand:
["('X',)", "('spade', '2')"]
Dealers Hand:
["('heart', '6')", "('spade', '2')"]
304
Player has 1 hands: 

["('heart', 'Q')", "('heart', '9')"]

Dealers Hand:
["('diamond', '5')", "('diamond', 'A')", "('diamond', 'K')"]
312


In [239]:
game.gameStatus(game.Player.hands[0])

1